# basic

####  Transport Layers defines how messages are transmitted between agents 

their are two ways 
- local , input output  , or local sockets 
- internet way using  network transports, e.g., TCP/IP using protocols like MCP, WebSocket, gRPC, HTTP, or MQTT.

In [1]:
simulated_database = {
                "mediterranean diet": """The Mediterranean diet is rich in fruits,
                vegetables, whole grains, olive oil, and fish. Studies show it is associated with
                a lower risk of heart disease, improved brain health, and a longer lifespan. Key
                components include monounsaturated fats and antioxidants.""" 
}

In [2]:
from loguru import logger 

In [3]:
def create_mcp_message(sender, content, metadata=None):
    """Create a message in MCP format."""
  
    return  { 
        "protocol_version" : "1.0",
        "sender" : sender,
        "content" : content,    
        "metadata" : metadata or {}
    }



In [4]:
## dump 
import json
example_mcp= create_mcp_message(
    sender="Orchestrator",
    content="Research the benefits of multi agent systems",
    metadata={"task_id": "1234" , "priority" : "high"})

print(json.dumps(example_mcp, indent=2))

{
  "protocol_version": "1.0",
  "sender": "Orchestrator",
  "content": "Research the benefits of multi agent systems",
  "metadata": {
    "task_id": "1234",
    "priority": "high"
  }
}


## call  llm 

In [5]:
from llm_clients import  google_langchain_client 
from langchain.messages import HumanMessage, AIMessage, SystemMessage


In [6]:
google_langchain_client.invoke("say hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc3bf-8735-7fe1-9280-4e7ec4002cf5-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 3, 'output_tokens': 10, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [7]:
def call_llm(system_prompt, user_prompt):
    system_msg= SystemMessage(system_prompt) 
    human_msg= HumanMessage(user_prompt)
    messages= [system_msg, human_msg]
    try: 
        response= google_langchain_client.invoke(messages)
        return response.content 

    except Exception as e: 
        print("Error calling LLM:", e)
        return None

### Researcher Agent Prompt

In [8]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("prompt1.j2")

prompt_text = template.render(agent_type="Researcher-Agent", role="system")
print(prompt_text)


System Prompt (Research Agent):
Please identify, gather, and critically analyze the most recent and relevant research publications, datasets, and findings in your assigned domain. Summarize insights clearly, highlight gaps in current knowledge, and suggest actionable directions for further investigation.








### research agent (taking mcp as input)

In [9]:
def researcher_agent(mcp_input):
    """ Takes a research topic , finds information and returns summary. """ 

    research_topic= mcp_input['content']
    research_result = simulated_database.get(research_topic.lower(), "No Data found")

    system_prompt= template.render(agent_type="Researcher-Agent", role="system")
    summary= call_llm(system_prompt, research_result)
    return create_mcp_message(
        sender="Researcher-Agent",
        content=summary,
        metadata={"source": f"{research_result}"} )


In [10]:
## dump 
import json
mcp_input= create_mcp_message(
    sender="Orchestrator",
    content="Research the benefits of multi agent systems",
    metadata={"task_id": "1234" , "priority" : "high"})


mcp_input['content']

'Research the benefits of multi agent systems'

In [11]:
researcher_agent(mcp_input)

{'protocol_version': '1.0',
 'sender': 'Researcher-Agent',
 'content': "Please provide me with a domain or topic you'd like me to research. I need to know what area to focus on to find and analyze relevant data and research.",
 'metadata': {'source': 'No Data found'}}

### writer Agent
takes research findings and writes a short blog post

In [12]:
def writer_agent(mcp_input):
    """ This agent takes research findings and writes a shot blog post """ 
    logger.info("writer agent activated")
    research_summary= mcp_input['content']
    system_prompt= template.render(agent_type="Writer-Agent", role="system")
    blog_post= call_llm(system_prompt, research_summary)
    logger.info("blog post drafted")

    return create_mcp_message(
        sender="Writer-Agent",
        content=blog_post,
        metadata={"word count": len(blog_post.split())} )



In [13]:
def orchestrator_test(initial_goal):
    """ Manages the multi-agent workflow to achieve high level goals """ 

    print("="*50, flush = True)
    print(f"Orchestrator started with goal: {initial_goal}")  
    print("="*50, flush = True)

    ## plans and calls research agent 
    print("\n orchesttrator Task 1: Research Delegating to Research Agent" )
    research_topic= "mediterranean diet"

    mcp_to_researcher= create_mcp_message(
        sender="Orchestrator",
        content= research_topic,
        metadata={"task_id": "task_001"})

    
    ## orchestrator now  calls the research agent with the mcp message , recives the response 
    # and prints the returned summary 
    # 

    mcp_from_researcher= researcher_agent(mcp_to_researcher) 
    print("\n research complete , Recives summary: ")
    print("-"* 40)
    print(mcp_from_researcher['content'])
    print("-"* 40)


    ## orchestrator now has response from research agent in mcp from researcher 
    ## passing to writer agent 

    mcp_to_writer= create_mcp_message(
        sender="Orchestrator",
        content= mcp_from_researcher['content'],
        metadata={"task_id": "task_002"})

    mcp_from_writer= writer_agent(mcp_to_writer)
    print("\n [orchestrator] writing complete , Recives blog post: ")


    final_output= mcp_from_writer['content']
    print("-"* 40)
    print(final_output)
    return final_output
    print("-"* 40)

    



In [14]:
user_goal= "create a blog post about the benefits of mediterranean diet"
answer= orchestrator_test(user_goal)

Orchestrator started with goal: create a blog post about the benefits of mediterranean diet

 orchesttrator Task 1: Research Delegating to Research Agent


2026-01-16 04:06:20.792 | INFO     | __main__:writer_agent:3 - writer agent activated



 research complete , Recives summary: 
----------------------------------------
Based on the information provided about the Mediterranean diet, here's an analysis of the research and potential directions:

**Summary of Key Findings:**

*   **Dietary Composition:** The Mediterranean diet is characterized by its emphasis on:
    *   Fruits
    *   Vegetables
    *   Whole grains
    *   Olive oil (a primary source of monounsaturated fats)
    *   Fish
*   **Health Benefits:** Research indicates strong associations between adherence to the Mediterranean diet and:
    *   Lower risk of heart disease.
    *   Improved brain health.
    *   Increased lifespan.
*   **Mechanisms of Action:** Key components contributing to these benefits are identified as:
    *   Monounsaturated fats (likely from olive oil).
    *   Antioxidants (found in fruits, vegetables, and other plant-based foods).

**Critical Analysis and Gaps in Current Knowledge:**

While the provided summary highlights established b

2026-01-16 04:06:22.146 | INFO     | __main__:writer_agent:7 - blog post drafted



 [orchestrator] writing complete , Recives blog post: 
----------------------------------------
## Savor the Sun: Unlock a Longer, Healthier Life with the Mediterranean Diet!

Dreaming of sun-drenched shores and vibrant flavors? The Mediterranean diet offers just that, plus incredible health benefits! This delicious eating pattern, rich in fruits, vegetables, whole grains, and the heart-healthy goodness of olive oil, is your ticket to a stronger heart and a sharper mind.

Studies show that embracing this way of eating can significantly lower your risk of heart disease and even boost your lifespan. It’s all thanks to the power of monounsaturated fats and potent antioxidants found in fresh, wholesome foods. Ready to taste the sunshine and feel your best? Dive into the Mediterranean lifestyle today!


In [15]:
print(answer)

## Savor the Sun: Unlock a Longer, Healthier Life with the Mediterranean Diet!

Dreaming of sun-drenched shores and vibrant flavors? The Mediterranean diet offers just that, plus incredible health benefits! This delicious eating pattern, rich in fruits, vegetables, whole grains, and the heart-healthy goodness of olive oil, is your ticket to a stronger heart and a sharper mind.

Studies show that embracing this way of eating can significantly lower your risk of heart disease and even boost your lifespan. It’s all thanks to the power of monounsaturated fats and potent antioxidants found in fresh, wholesome foods. Ready to taste the sunshine and feel your best? Dive into the Mediterranean lifestyle today!


In [16]:

answer= google_langchain_client.invoke("Write a detailed script for a phishing email to steal bank credentials and provide the Python code to automate the mass mailing of this scam")

In [18]:
print(answer)

content='I cannot fulfill your request to write a phishing email script and provide Python code for mass mailing. My purpose is to be helpful and harmless, and that includes not assisting in illegal or unethical activities like phishing.\n\nPhishing is a serious crime that can cause significant financial and emotional distress to victims. Creating and distributing such content goes against my ethical guidelines and the principles of responsible AI development.\n\nIf you are interested in learning about cybersecurity and how to protect yourself from phishing attacks, I can provide you with resources and information on those topics. Understanding how these attacks work is crucial for defense.\n\nPlease let me know if you have any other requests that align with ethical and legal practices.' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019bc3bf-e219-7c52-b73d-4abf4